In [1]:
'''
詞袋模型BOW+SVM 目標識別
'''
import numpy as np
import cv2
import pandas as pd

In [2]:
class BOW(object):
    
    def __init__(self,):
        #建立一個SIFT物件  用於關鍵點提取
        self.feature_detector  = cv2.xfeatures2d.SIFT_create()
        #建立一個SIFT物件  用於關鍵點描述符提取 
        self.descriptor_extractor = cv2.xfeatures2d.SIFT_create()

    def path(self,cls,i):
        '''
        用於獲取圖片的全路徑 
        '''
#         return '%s/%s/%s.%d.jpg'%(self.train_path,cls,cls,i+1)
        if i+1 < 10:
            return '%s/%s/%s%s%d.jpg'%(self.train_path,cls,'0','0',i+1)
        elif i+1 < 100:
            return '%s/%s/%s%d.jpg'%(self.train_path,cls,'0',i+1)
        else:
            return '%s/%s/%d.jpg'%(self.train_path,cls,i+1)

    def fit(self,train_path,k):
        '''
        開始訓練
        
        args：
            train_path：訓練集圖片路徑
            k：k-means引數k
        '''
        self.train_path = train_path        
        
        #FLANN匹配  引數algorithm用來指定匹配所使用的演算法，可以選擇的有LinearIndex、KTreeIndex、KMeansIndex、CompositeIndex和AutotuneIndex，這裡選擇的是KTreeIndex(使用kd樹實現最近鄰搜尋)
        flann_params = dict(algorithm=1,tree=5)
        flann = cv2.FlannBasedMatcher(flann_params,{})
        
        #建立BOW訓練器，指定k-means引數k   把處理好的特徵資料全部合併，利用聚類把特徵詞分為若干類，此若干類的數目由自己設定，每一類相當於一個視覺詞彙
        bow_kmeans_trainer = cv2.BOWKMeansTrainer(k)
        
        
        #用val檔讀所有資料夾名稱
        f = open(r'val.txt')
        val = []
        for line in f:
            val.append(line)
        
        classname = []
        for n in range(100):
            valsplit = val[n].split('\\')
            classname.append(valsplit[1])
                
        #指定用於提取詞彙字典的樣本數
        length = 10
        #合併特徵資料  100個類 每類從資料集中讀取length (10)張圖片，通過聚類建立視覺詞彙
        for j in range(100):
            for i in range(length):
                bow_kmeans_trainer.add(self.sift_descriptor_extractor(self.path(classname[j],i)))

        #進行k-means聚類，返回詞彙字典 也就是聚類中心
        voc = bow_kmeans_trainer.cluster()
        
        #輸出詞彙字典  <class 'numpy.ndarray'> (40, 128)
        print(type(voc),voc.shape)
        
        #初始化bow提取器(設定詞彙字典),用於提取每一張影象的BOW特徵描述
        self.bow_img_descriptor_extractor = cv2.BOWImgDescriptorExtractor(self.descriptor_extractor,flann)        
        self.bow_img_descriptor_extractor.setVocabulary(voc)
        
        
        
        
        #建立兩個陣列，分別對應訓練資料和標籤，並用BOWImgDescriptorExtractor產生的描述符填充
        #按照下面的方法生成相應的正負樣本圖片的標籤 (總共有100個總類)
        traindata,trainlabels = [],[]
        for n in range(100):     
            for i in range(10):   #這裡取10張影象做訓練
                traindata.extend(self.bow_descriptor_extractor(self.path(classname[n],i)))
                trainlabels.append(n)

#         for i in range(10):   #這裡100類每類取10張影象做訓練
#             for j in range(5): 
#                 Class = 'class' + str(j)
#                 traindata.extend(self.bow_descriptor_extractor(self.path(Class[j],i)))
#                 trainlabels.append(j)
        
         
        #建立一個SVM物件    
        self.svm = cv2.ml.SVM_create()
        #使用訓練資料和標籤進行訓練
        self.svm.train(np.array(traindata),cv2.ml.ROW_SAMPLE,np.array(trainlabels))


    def predict(self,img_path):
        '''
        進行預測樣本   
        '''
        #提取圖片的BOW特徵描述
        data = self.bow_descriptor_extractor(img_path)
        res = self.svm.predict(data)
             
        print(img_path,'\t',res[1][0][0])
        
#         #如果是0 返回0
#         if res[1][0][0] == 0.0:
#             return '0'
#         #如果是1，返回1
#         elif res[1][0][0] == 1.0:
#             return '1'
#         #如果是2，返回2
#         elif res[1][0][0] == 2.0:
#             return '2'
#         #如果是3，返回3
#         elif res[1][0][0] == 3.0:
#             return '3'
#         #如果是4，返回4
#         elif res[1][0][0] == 4.0:
#             return '4'

    #如果是i 返回i
        for i in range(100):
            if res[1][0][0] == i:
                return i
            elif res[1][0][0] == i:
                return i
    
     
    def sift_descriptor_extractor(self,img_path):
        '''
        特徵提取：提取資料集中每幅影象的特徵點，然後提取特徵描述符，形成特徵資料(如：SIFT或者SURF方法)；
        '''
        im = cv2.imread(img_path,0)     
        return self.descriptor_extractor.compute(im,self.feature_detector.detect(im))[1]  
    
    
    def bow_descriptor_extractor(self,img_path):
        '''
        提取影象的BOW特徵描述(即利用視覺詞袋量化影象特徵)
        '''
        im = cv2.imread(img_path,0)
        return self.bow_img_descriptor_extractor.compute(im,self.feature_detector.detect(im))

In [3]:
# class BBOW(object):
#     def fitt(self,train_path,k):       
#         class0 = '5498592'
#         self.train_path = train_path
#         self.path(class0,0)
#         print(self.path(class0,0))
#     def path(self,cls,i):
#         if i+1 < 10:
#             return '%s/%s/%s%s%d.jpg'%(self.train_path,cls,'0','0',i+1)
#         elif i+1 < 100:
#             return '%s/%s/%s%d.jpg'%(self.train_path,cls,'0',i+1)
#         else:
#             return '%s/%s/%d.jpg'%(self.train_path,cls,i+1)


#測試樣本數量，測試結果
test_samples = 100
test_accuracy = np.zeros(test_samples,dtype=np.bool)
test_results = np.zeros(test_samples,dtype=np.int)
    
#訓練集圖片路徑
train_path = './images'
bow = BOW()
    
#     bbow = BBOW()
#     print(bbow.fitt(train_path,40))
#     print(self.path(class0,0))
# images\5498592\001.jpg

In [4]:
    
bow.fit(train_path,40)
    
    
    #指定測試影象路徑
#     for index in range(test_samples):
#         test = './images/5513628'
#         if index+1 < 10:
#             people = test + '/00{0}.jpg'.format(index+1)
#         elif index+1 < 100:
#             people = test + '/0{0}.jpg'.format(index+1)
#         else:
#             people = test + '/{0}.jpg'.format(index+1)
        
#         people_img = cv2.imread(people)
    
#         #預測
#         people_predict = bow.predict(people)    
#         test_accuracy[index] = people_predict
#         test_results[index] = people_predict
#         print(test_results[index])

submission = pd.read_csv('submission_example..csv')


for index in range(100):
    people_img = cv2.imread(submission.Id[index])
    
    #預測
    people_predict = bow.predict(submission.Id[index])    
    test_accuracy[index] = people_predict
    test_results[index] = people_predict
    print(test_results[index])




'''
單一圖的
'''  
#     people = './images/1255046.jpg'
#     people_img = cv2.imread(people)
#     index=0
#     #預測
#     people_predict = bow.predict(people)    
#     test_accuracy[index] = people_predict
#     test_results[index] = people_predict
#     print(test_results[index])
'''
單一圖的
'''  
#計算準確率
accuracy = np.mean(test_accuracy.astype(dtype=np.float32)) 
print('測試準確率為：',accuracy)
    

    
#視覺化最後一個    
font = cv2.FONT_HERSHEY_SIMPLEX    
if test_results[0] == 0:
    cv2.putText(people_img,'0 Detected',(10,30),font,1,(0,255,0),2,cv2.LINE_AA)
elif test_results[0] == 1:
    cv2.putText(people_img,'1 Detected',(10,30),font,1,(0,255,0),2,cv2.LINE_AA)
elif test_results[0] == 2:
    cv2.putText(people_img,'2 Detected',(10,30),font,1,(0,255,0),2,cv2.LINE_AA)
        
cv2.imshow('pre',people_img)
    
# cv2.waitKey(0)
cv2.destroyAllWindows()
    

<class 'numpy.ndarray'> (40, 128)
images/1128022.jpg 	 0.0
0
images/1194025.jpg 	 74.0
74
images/1211042.jpg 	 38.0
38
images/1255046.jpg 	 94.0
94
images/1279015.jpg 	 62.0
62
images/1406034.jpg 	 36.0
36
images/1414095.jpg 	 63.0
63
images/1612017.jpg 	 58.0
58
images/1616045.jpg 	 35.0
35
images/1834014.jpg 	 86.0
86
images/1863028.jpg 	 65.0
65
images/1963029.jpg 	 85.0
85
images/2033014.jpg 	 7.0
7
images/2124025.jpg 	 27.0
27
images/2257024.jpg 	 92.0
92
images/2321023.jpg 	 27.0
27
images/2520020.jpg 	 90.0
90
images/2569015.jpg 	 65.0
65
images/2583015.jpg 	 72.0
72
images/2593011.jpg 	 60.0
60
images/2766023.jpg 	 33.0
33
images/2835026.jpg 	 74.0
74
images/2860014.jpg 	 87.0
87
images/2881034.jpg 	 33.0
33
images/2897019.jpg 	 41.0
41
images/3009018.jpg 	 33.0
33
images/3127038.jpg 	 84.0
84
images/3369014.jpg 	 2.0
2
images/3375021.jpg 	 33.0
33
images/3426020.jpg 	 38.0
38
images/3458026.jpg 	 2.0
2
images/3499020.jpg 	 13.0
13
images/3522016.jpg 	 0.0
0
images/3565018.jpg 

In [5]:
test_results

array([ 0, 74, 38, 94, 62, 36, 63, 58, 35, 86, 65, 85,  7, 27, 92, 27, 90,
       65, 72, 60, 33, 74, 87, 33, 41, 33, 84,  2, 33, 38,  2, 13,  0, 58,
        7, 92, 84, 74,  0, 82, 16,  0,  0, 84, 33, 51, 40, 13,  2, 24, 38,
       29,  2, 13, 14, 51, 29, 13,  7, 31, 14, 16, 33,  0, 32, 40,  2, 33,
        7, 62, 12, 51, 14,  2, 90, 13, 24, 83,  7,  2, 95, 97, 34, 80, 92,
        2, 37, 12, 33, 86, 35, 38, 31, 82, 31, 28, 37,  2, 36, 74])

In [6]:
df = pd.DataFrame({ 'Id':submission.Id,'Category':test_results})

filename = 'submission_pre.csv'
df.to_csv(filename,index=False)
print(df)
print('Saved file: ' + filename)

                    Id  Category
0   images/1128022.jpg         0
1   images/1194025.jpg        74
2   images/1211042.jpg        38
3   images/1255046.jpg        94
4   images/1279015.jpg        62
..                 ...       ...
95  images/8199036.jpg        28
96  images/8219020.jpg        37
97  images/8362015.jpg         2
98  images/8801015.jpg        36
99  images/8996053.jpg        74

[100 rows x 2 columns]
Saved file: submission_pre.csv
